In [1]:
import sys 
sys.path.append('..')
from utils.paths import make_dirs, QUARTERS_DICT, HASHTAG_TOP_DIR, HASHTAG_FREQUENCY_DIR_ALL, POTATOES_CLEANED_DIR, FORBIDDEN_HASHTAGS_LIST, POTATOES_HASHTAGS_CLEAN, QUARTER_RESULTS, POTATOES_DIR, QUARTERS_ALL_POSTS, HASHTAGS_PER_USER_DIR, HASHTAG_FREQUENCY_DIR, CONNECTED_COMPONENTS_DIR, CONNECTED_COMPONENTS_PUBLIC_SPACES_DIR, HASHTAG_FREQUENCY_PUBLIC_SPACES_DIR, HASHTAGS_PER_USER_PUBLIC_SPACES_DIR
from utils.utils import load_dataframes, load_dataframe, write_dict_to_csv, write_list_to_csv, write_df_to_csv
import pandas as pd
from frequency.frequency import count_hashtag_frequency, group_by_user
import os

make_dirs()

min_count = 10

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sophiaknapp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sophiaknapp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Rank

rank * count

In [19]:

# THE RESULTS OF THIS DONT MEAN ANYTHING
dict = QUARTERS_DICT()
for quarter in dict:
    print('>> now processing', quarter)
    df = dict[quarter]
    df = group_by_user(df)
    write_df_to_csv(df, quarter, HASHTAGS_PER_USER_DIR)
    fdist = count_hashtag_frequency(df)
    df = pd.DataFrame(fdist, columns=['hashtag', 'count'])
    df['rank'] = df['count'].rank()
    df['rank'] = df['rank'] * df['count']
    write_df_to_csv(df, quarter, HASHTAG_FREQUENCY_DIR)


>> now loading: allach
>> now loading: altperlach
>> now loading: altstadt
>> now loading: amhart
>> now loading: arabellapark
>> now loading: au
>> now loading: aubing
>> now loading: bergamlaim
>> now loading: bogenhausen
>> now loading: borstei
>> now loading: daglfing
>> now loading: denning
>> now loading: fasanerie
>> now loading: fasangarten
>> now loading: feldmoching
>> now loading: forstenried
>> now loading: freiham
>> now loading: freimann
>> now loading: fröttmaning
>> now loading: fürstenried
>> now loading: giesing
>> now loading: hadern
>> now loading: haidhausen
>> now loading: harlaching
>> now loading: harras
>> now loading: harthof
>> now loading: hasenbergl
>> now loading: hellabrunn
>> now loading: herzogpark
>> now loading: isarvorstadt
>> now loading: johanneskirchen
>> now loading: kieferngarten
>> now loading: kirchtrudering
>> now loading: laim
>> now loading: lehel
>> now loading: lerchenau
>> now loading: lochhausen
>> now loading: ludwigsvorstadt
>> now lo

Get average rank for whole city

In [2]:
# THE RESULTS OF THIS DONT MEAN ANYTHING
# COULD DO: normalize rank
def get_average_rank(dfs, min_count, out_dir):
    df = pd.DataFrame()
    df['hashtag'] = [] 
    df = df.set_index('hashtag')

    idx = 1

    for quarter in dfs:
        print ('file no:', idx)
        idx += 1
        print ('>> now processing:', quarter)
        df_new = dfs[quarter]
        df_new.drop(df_new[df_new['count'] < min_count].index, inplace = True)
        df = pd.merge(df, df_new, on='hashtag', how='outer')

    df = df.fillna(0)

    rank_columns = [column for column in df.columns if 'rank' in column]
    count_columns = [column for column in df.columns if 'count' in column]
    
    df['rank_average'] = df[rank_columns].apply(
      sum,
      axis=1
    )
    df['rank_average'] = df['rank_average']/len(df)

    df['count_all'] = df[count_columns].apply(
      sum,
      axis=1
    )
    df = df[['hashtag', 'count_all', 'rank_average']]

    df.sort_values(by=['rank_average'], ascending=False, inplace=True)
    write_df_to_csv(df,'ordered_by_rank', out_dir)
    df.sort_values(by=['count_all'], ascending=False, inplace=True)
    write_df_to_csv(df,'ordered_by_count', out_dir)
    return

In [3]:
dfs = load_dataframes(HASHTAG_FREQUENCY_DIR)
get_average_rank(dfs, min_count, HASHTAG_FREQUENCY_DIR_ALL)

>> now loading: allach
>> now loading: altperlach
>> now loading: altstadt
>> now loading: amhart
>> now loading: arabellapark
>> now loading: au
>> now loading: aubing
>> now loading: bergamlaim
>> now loading: bogenhausen
>> now loading: borstei
>> now loading: daglfing
>> now loading: denning
>> now loading: fasanerie
>> now loading: fasangarten
>> now loading: feldmoching
>> now loading: forstenried
>> now loading: freiham
>> now loading: freimann
>> now loading: fröttmaning
>> now loading: fürstenried
>> now loading: giesing
>> now loading: hadern
>> now loading: haidhausen
>> now loading: harlaching
>> now loading: harras
>> now loading: harthof
>> now loading: hasenbergl
>> now loading: hellabrunn
>> now loading: herzogpark
>> now loading: isarvorstadt
>> now loading: johanneskirchen
>> now loading: kieferngarten
>> now loading: kirchtrudering
>> now loading: laim
>> now loading: lehel
>> now loading: lerchenau
>> now loading: lochhausen
>> now loading: ludwigsvorstadt
>> now lo

Get top flop

In [8]:
def get_rank_diff(dfs, df_all, min_count, out_dir):
    idx = 1

    for quarter in dfs:
        print ('file no:', idx)
        idx += 1
        print ('>> now processing:', quarter)

        df = dfs[quarter]
        df.drop(df[df['count'] < min_count].index, inplace = True)

        df = pd.merge(df, df_all, on='hashtag', how='outer')

        df = df.fillna(0)
        df['rank_diff'] = df['rank'] - df['rank_average']


        df_top = df[df['rank_diff']>0]
        df_top.sort_values(by=['rank_diff'], ascending=False)

        print(df_top[:10])
        # write_df_to_csv(df, quarter, out_dir)
        write_df_to_csv(df_top, quarter + '_top', out_dir)


        df_flop = df[df['rank_diff']<0]
        df_flop.sort_values(by=['rank_diff'], ascending=False)
        write_df_to_csv(df_flop, quarter + '_flop', out_dir)

    return

In [10]:
dfs = load_dataframes(HASHTAG_FREQUENCY_DIR)
df_all = load_dataframe(HASHTAG_FREQUENCY_DIR_ALL, 'ordered_by_rank')

get_rank_diff(dfs, df_all, min_count, HASHTAG_TOP_DIR)

>> now loading: allach
>> now loading: altperlach
>> now loading: altstadt
>> now loading: amhart
>> now loading: arabellapark
>> now loading: au
>> now loading: aubing
>> now loading: bergamlaim
>> now loading: bogenhausen
>> now loading: borstei
>> now loading: daglfing
>> now loading: denning
>> now loading: fasanerie
>> now loading: fasangarten
>> now loading: feldmoching
>> now loading: forstenried
>> now loading: freiham
>> now loading: freimann
>> now loading: fröttmaning
>> now loading: fürstenried
>> now loading: giesing
>> now loading: hadern
>> now loading: haidhausen
>> now loading: harlaching
>> now loading: harras
>> now loading: harthof
>> now loading: hasenbergl
>> now loading: hellabrunn
>> now loading: herzogpark
>> now loading: isarvorstadt
>> now loading: johanneskirchen
>> now loading: kieferngarten
>> now loading: kirchtrudering
>> now loading: laim
>> now loading: lehel
>> now loading: lerchenau
>> now loading: lochhausen
>> now loading: ludwigsvorstadt
>> now lo